In [1]:
import asyncio
from prompt_test.utils import send_chat_completion, send_async_requests
from prompt_test.data import has_sentences, load_full_ragbench
BATCH_SIZE = 8




/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import pickle
# with open('../tmp/emanual.test.pkl', 'rb') as f:
#     t = pickle.load(f)
# t

In [3]:
ragbench = load_full_ragbench()
ragbench

{'msmarco': DatasetDict({
     train: Dataset({
         features: ['id', 'question', 'documents', 'response', 'generation_model_name', 'annotating_model_name', 'dataset_name', 'documents_sentences', 'response_sentences', 'sentence_support_information', 'unsupported_response_sentence_keys', 'adherence_score', 'overall_supported_explanation', 'relevance_explanation', 'all_relevant_sentence_keys', 'all_utilized_sentence_keys', 'trulens_groundedness', 'trulens_context_relevance', 'ragas_faithfulness', 'ragas_context_relevance', 'gpt3_adherence', 'gpt3_context_relevance', 'gpt35_utilization', 'relevance_score', 'utilization_score', 'completeness_score'],
         num_rows: 1870
     })
     test: Dataset({
         features: ['id', 'question', 'documents', 'response', 'generation_model_name', 'annotating_model_name', 'dataset_name', 'documents_sentences', 'response_sentences', 'sentence_support_information', 'unsupported_response_sentence_keys', 'adherence_score', 'overall_supported_explan

In [4]:
from transformers import AutoTokenizer
MODEL_NAME = "Qwen/Qwen2.5-72B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
from openai import AsyncOpenAI
client = AsyncOpenAI(api_key='874c364705747e7ab314ceba89c2029c9a72ab2154664c470eb4ce18c2f0acb0', base_url= "http://10.36.60.52:1234/v1")
model_id = 'qwen2.5-72b'

In [ ]:
res = await send_async_requests([[
                {"role": "system", "content": 'Ты полезный бот'},
                {"role": "user", "content": 'напиши стих'}
            ]]*3, model=model_id, client=client,max_tokens=2048)
res

In [ ]:
def make_messages(item, prompt="Ты — профессиональный переводчик с английского на русский. Переводи точно и понятно."):
    documents = ""
    for doc in item['documents_sentences']:
        for sentence in doc:
            documents += " " + " ".join(sentence)
        documents += "\n"
    text = item['question'] + documents + item['response']
#     print(text)
    return [
                {"role": "system", "content": prompt},
                {"role": "user", "content": text}
            ]

def under_token_limit(item, prompt, tokenizer, max_tokens=8000):
    messages = make_messages(item)
    encoded = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        truncation=False,
        return_tensors="pt"
    )
    # shape == (1, seq_len)
    seq_len = encoded.shape[1]
    return seq_len <= max_tokens


In [ ]:
ragbench

In [ ]:
from datasets import DatasetDict
ragbench_new = {}

for name, dataset in ragbench.items():
    ragbench_new[name] = {}  
    for subset in ['train', 'validation', 'test']:
        if subset in dataset:
            print(f'Датасет {name}.{subset}: {len(dataset[subset])} элементов')

            filtered = dataset[subset].filter(has_sentences)
            print(f'Датасет {name}.{subset} после фильтрации has_sentences: {len(filtered)} элементов')

            filtered = filtered.filter(lambda x: under_token_limit(x, None, tokenizer, max_tokens=8000))
            print(f'Датасет {name}.{subset} после фильтрации по max_tokens: {len(filtered)} элементов')
#             filtered = filtered.shuffle(seed=42).select(range(16))
            ragbench_new[name][subset] = filtered
for ds_name, splits in ragbench_new.items():
    ragbench_new[ds_name] = DatasetDict(splits)

In [ ]:
ragbench_new

In [ ]:
async def send_chat_completion_batch(batch_messages, model, client,
                                     max_tokens=256, temperature=0.7, top_p=0.9, top_k=50):
    extra_body = {
        "top_k": top_k,
        "top_p": top_p,
        "temperature": temperature
    }
    try:
        completion = await client.chat.completions.create(
            model=model,
            messages=batch_messages,
            max_tokens=max_tokens,
            seed=42,
            extra_body=extra_body
        )
    except Exception as e:
        try:
            completion = await client.chat.completions.create(
                model=model,
                messages=batch_messages,
                max_tokens=max_tokens,
                seed=42,
                extra_body=extra_body
            )
        except Exception as e:
            try:
                completion = await client.chat.completions.create(
                    model=model,
                    messages=batch_messages,
                    max_tokens=max_tokens,
                    seed=42,
                    extra_body=extra_body
                )
            except Exception as e:
                raise e
    return [choice.message.content.strip() for choice in completion.choices]

In [ ]:
async def translate_texts_in_batches(texts, model, client, max_tokens=256, temperature=0.7, top_p=0.9, top_k=50):
    all_translations = []

    for i in range(0, len(texts), BATCH_SIZE):
        batch_texts = texts[i:i + BATCH_SIZE]

        batch_messages = [
            [
                {"role": "system", "content": "Ты — профессиональный переводчик с английского на русский. Переводи точно и понятно. Рассуждения и тд не нужны, лишь перевод."},
                {"role": "user", "content": text}
            ]
            for text in batch_texts
        ]
#         print(batch_messages)
        batch_translations = await send_async_requests(
            batch_messages,
            model,
            client,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k
        )
#         print(batch_translations)
        all_translations.extend(batch_translations)

    return all_translations

def flatten_documents_sentences(documents_sentences):
    texts = []
    for doc_sentences in documents_sentences:
        for sentence in doc_sentences:
            # sentence пример: ['0a', 'Title: Emergent severe acute respiratory distress syndrome caused by adenovirus type 55 ...']
            texts.append(sentence[1])
    return texts

def rebuild_documents_sentences_with_translations(documents_sentences, translations):
    new_documents_sentences = []
    idx = 0
    for doc_sentences in documents_sentences:
        new_doc = []
        for sentence in doc_sentences:
            number = sentence[0]
            translated_text = translations[idx]
            idx += 1
            new_doc.append([number, translated_text])
        new_documents_sentences.append(new_doc)
    return new_documents_sentences

async def translate_all_datasets(ragbench, text_fields, new_column_names, model, client):

    updated_ragbench = {}

    for dataset_name, dataset_splits in ragbench.items():
        print(f"Обрабатываем датасет: {dataset_name}")
        updated_ragbench[dataset_name] = {}

        for split_name, split_dataset in dataset_splits.items():
            print(f"  Сплит: {split_name} — элементов: {len(split_dataset)}")
            questions = [item['question'] for item in split_dataset]
            responses = [item['response'] for item in split_dataset]

            all_docs_texts = []
            docs_indices = []  
            for i, item in enumerate(split_dataset):
                docs_texts = flatten_documents_sentences(item['documents_sentences'])
                all_docs_texts.extend(docs_texts)
                docs_indices.append((i, len(docs_texts)))

            print("  Перевод вопросов...")
            questions_translated = await translate_texts_in_batches(questions, model, client)
            print("  Перевод ответов...")
            responses_translated = await translate_texts_in_batches(responses, model, client)
            print("  Перевод предложений из документов...")
            docs_translated = await translate_texts_in_batches(all_docs_texts, model, client)

            new_documents_sentences = [[] for _ in range(len(split_dataset))]
            idx = 0
            for elem_idx, length in docs_indices:
                translated_sents = docs_translated[idx: idx + length]
                idx += length
                new_documents_sentences[elem_idx] = rebuild_documents_sentences_with_translations(
                    split_dataset[elem_idx]['documents_sentences'],
                    translated_sents
                )

            new_items = []
            for i, item in enumerate(split_dataset):
                new_item = dict(item)
                new_item[new_column_names[0]] = questions_translated[i]
                new_item[new_column_names[1]] = responses_translated[i]
                new_item[new_column_names[2]] = new_documents_sentences[i]
                new_items.append(new_item)

            from datasets import Dataset
            updated_split = Dataset.from_list(new_items)
            updated_ragbench[dataset_name][split_name] = updated_split

    return updated_ragbench

In [ ]:
import os
import json
from datasets import Dataset
from datasets import load_dataset
BATCH_SIZE = 8

def batched(iterable, n):
    for i in range(0, len(iterable), n):
        yield iterable[i:i+n]

async def translate_all_datasets_chunked(ragbench, text_fields, new_column_names, model, client, ckpt_dir="checkpoints"):
    os.makedirs(ckpt_dir, exist_ok=True)
    updated_ragbench = {}

    for dataset_name, dataset_splits in ragbench.items():
        print(f"Обрабатываем датасет: {dataset_name}")
        updated_ragbench[dataset_name] = {}

        for split_name, split_dataset in dataset_splits.items():
            print(f"  Сплит: {split_name} — элементов: {len(split_dataset)}")

            ckpt_path = os.path.join(ckpt_dir, f"{dataset_name}_{split_name}.jsonl")
            processed_items = []
            if os.path.exists(ckpt_path):
                with open(ckpt_path, "r", encoding="utf-8") as f:
                    for line in f:
                        processed_items.append(json.loads(line))
                print(f"    Найден checkpoint: {len(processed_items)} элементов уже обработано")
            else:
                print("    Нет checkpoint, начинаем с начала")

            N_done = len(processed_items)

            if N_done == len(split_dataset):
                print("    Все элементы уже обработаны, загружаем из checkpoint")
                ds = Dataset.from_list(processed_items)
                updated_ragbench[dataset_name][split_name] = ds
                continue

            with open(ckpt_path, "a", encoding="utf-8") as f:

                for batch_start in range(N_done, len(split_dataset), BATCH_SIZE):
                    batch_end = min(batch_start + BATCH_SIZE, len(split_dataset))
                    batch = split_dataset[batch_start:batch_end]
#                     print(batch)
                    questions =  batch['question'] #[item['question'] for item in batch]
#                     print()
#                     questions_translated = await send_async_requests(
#                         questions,
#                         model,
#                         client,
#                         max_tokens=2048,
#                         temperature=0.7,
#                         top_p=0.9,
#                         top_k=50
#                     )
                    questions_translated = await translate_texts_in_batches(questions, model, client)
#                     print(questions_translated)
                    responses = batch['response'] #[item['response'] for item in batch]
#                     responses_translated = await send_async_requests(
#                         responses,
#                         model,
#                         client,
#                         max_tokens=2048,
#                         temperature=0.7,
#                         top_p=0.9,
#                         top_k=50
#                     )
        
                    responses_translated = await translate_texts_in_batches(responses, model, client)
                    all_docs_texts = []
                    docs_indices = []

                    batch_size = len(batch['id'])
                    for i in range(batch_size):
                        docs_texts = flatten_documents_sentences(batch['documents_sentences'][i])
                        all_docs_texts.extend(docs_texts)
                        docs_indices.append((i, len(docs_texts)))
#                     print(docs_indices, all_docs_texts)
                    docs_translated = await translate_texts_in_batches(all_docs_texts, model, client)

                    new_documents_sentences = [[] for _ in range(len(batch['id']))]
                    idx = 0
                    for elem_idx, length in docs_indices:
                        translated_sents = docs_translated[idx: idx + length]
                        idx += length
                        new_documents_sentences[elem_idx] = rebuild_documents_sentences_with_translations(
                            batch['documents_sentences'][elem_idx],
                            translated_sents
                        )

                    for i in range(len(batch['id'])):
                        new_item = {key: batch[key][i] for key in batch.keys()}

                        new_item[new_column_names[0]] = questions_translated[i]
                        new_item[new_column_names[1]] = responses_translated[i]
                        new_item[new_column_names[2]] = new_documents_sentences[i]

                        f.write(json.dumps(new_item, ensure_ascii=False) + "\n")
                        f.flush()
                        os.fsync(f.fileno())

#                         print(f"Сохранён элемент {batch_start + i + 1}/{len(split_dataset)}")
#                     break
            ds = load_dataset(
                "json",
                data_files={split_name: ckpt_path},
                split=split_name
            )
            updated_ragbench[dataset_name][split_name] = ds

    return updated_ragbench


In [ ]:
async def main():
    translated_ragbench = await translate_all_datasets_chunked(
        ragbench_new,
        text_fields=['question', 'response', 'documents_sentences'],
        new_column_names=['question_ru', 'response_ru', 'documents_sentences_ru'],
        model=model_id,
        client=client
    )
    return translated_ragbench


translated_ragbench = await main()

In [14]:
def sampled(bench, idx, dataset, fold, ru_field, field):
    return bench[dataset][fold][idx][ru_field], bench[dataset][fold][idx][field]


In [ ]:
translated_ragbench

In [ ]:
sampled(translated_ragbench,8, 'hagrid', 'train', 'response_ru', 'response')

In [15]:
# import pickle
# with open('../hagrid_ru.pkl', 'wb') as f:
#     pickle.dump(translated_ragbench, f)

In [13]:
import pickle
with open('../delucionqa_ru.pkl', 'rb') as f:
    translated_ragbench = pickle.load(f)
translated_ragbench

{'delucionqa': {'train': Dataset({
      features: ['id', 'question', 'documents', 'response', 'generation_model_name', 'annotating_model_name', 'dataset_name', 'documents_sentences', 'response_sentences', 'sentence_support_information', 'unsupported_response_sentence_keys', 'adherence_score', 'overall_supported_explanation', 'relevance_explanation', 'all_relevant_sentence_keys', 'all_utilized_sentence_keys', 'trulens_groundedness', 'trulens_context_relevance', 'ragas_faithfulness', 'ragas_context_relevance', 'gpt3_adherence', 'gpt3_context_relevance', 'gpt35_utilization', 'relevance_score', 'utilization_score', 'completeness_score', 'question_ru', 'response_ru', 'documents_sentences_ru'],
      num_rows: 1458
  }),
  'validation': Dataset({
      features: ['id', 'question', 'documents', 'response', 'generation_model_name', 'annotating_model_name', 'dataset_name', 'documents_sentences', 'response_sentences', 'sentence_support_information', 'unsupported_response_sentence_keys', 'adhere